# TMA4320 Introduksjon til vitenskapelige beregninger
## Øving 2

**Veiledning**: Onsdagene 08.02 og 15.02 kl. 16:15-18:00 i R50, Realfagsbygget, og torsdagene 09.02 og 16.02 i 14:15-16:00 i rom 256 SB1.  
**Innlevering**: Mandag 20.02 kl. 23:59, i [ovsys](https://ovsys.math.ntnu.no).

Oppgaven skal innleveres som et Jupyternotat. Men gjør gjerne implementering og koding i Spyder eller et annet IDE, og kopier den ferdige koden inn i Jupyternotatet for innlevering.

*Bare de første fire oppgavene skal leveres inn.*

**NB!** Før innlevering: 
* Kjør en runde på hele notatet for å se at alt virker: <tt>Kernel -> Restart & Run All</tt>
* Deretter: <tt>Kernel -> Restart & Clear Output</tt>. Fila er nå klar for innlevering. 

$\newcommand{mb}[1]{\mathbf{#1}}$
$\newcommand{R}{\mathbb{R}}$

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import lu, solve
from numpy.testing import assert_array_almost_equal
newparams = {'figure.figsize': (8.0, 4.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
plt.rcParams.update(newparams)

### Oppgave 1
Gitt matrisen
$$
A = \begin{bmatrix} 
    4 & -8 & 2  \\ 5 & 0 & 10 \\  2 & 2 & 1 
\end{bmatrix} 
$$
**(a)** Finn $LU$-faktoriseringen $A=L U$ for denne matrisen, der $L$ er nedre triangulær med 1-ere på diagonalen, og $U$ er øvre triangulær. 

**(b)** Bruk skalert delvis pivotering, og finn pivoteringsmatrisa $P$, $L$ og $U$ slik at 
$PA=LU$
der $P$ er pivoteringsmatrisen. 



In [2]:
# Oppgave 1a
def lu_fac_native(A):
    """
    Calculate the LU factorization of a matrix A such that A = LU.

    Parameters
    ----------
    A : np.ndarray (M, N)
        Matrix with real elements to factorize.

    Returns
    -------
    L : np.ndarray (M, K)
        Lower triangular unit matrix.
    U : np.ndarray (K, N)
        Upper triangular matrix.
    """
    U = np.array(A, copy=True, dtype=float)

    assert U.shape[0] == U.shape[1], 'A is not quadratic'

    n, _ = U.shape
    L = np.eye(n, n)

    for k in range(0, n-1):
        for i in range(k+1, n):
            m_ik = U[i,k] / U[k,k]
            L[i,k] += m_ik

            for j in range(k, n):
                U[i,j] -= m_ik * U[k,j]

    return L, U

A = np.asarray([
    [4, -8, 2],
    [5, 0, 10],
    [2, 2, 1]
], dtype=float)

L, U = lu_fac_native(A)
assert_array_almost_equal(L@U, A)
print(f'Task 1a\nL = \n{L}\nU = \n{U}')

Task 1a
L = 
[[1.   0.   0.  ]
 [1.25 1.   0.  ]
 [0.5  0.6  1.  ]]
U = 
[[ 4.  -8.   2. ]
 [ 0.  10.   7.5]
 [ 0.   0.  -4.5]]


In [3]:
# Oppgave 1b
def lu_fac_pivot(A):
    """
    Calculate the LU factorization of a matrix A such that PA = LU.

    Parameters
    ----------
    A : np.ndarray (M, N)
        Matrix with real elements to factorize.

    Returns
    -------
    P : np.ndarray (M, M)
        Permutation matrix
    L : np.ndarray (M, K)
        Lower triangular unit matrix.
    U : np.ndarray (K, N)
        Upper triangular matrix.
    """
    U = np.array(A, copy=True, dtype=float)

    assert U.shape[0] == U.shape[1], 'A is not quadratic'

    n, _ = U.shape
    L = np.eye(n, n)
    s = np.max(np.abs(A), axis=1)  # Scaling vector
    p = np.arange(n)  # Permutation order vector

    for i in range(n):
        # Finding max element in row
        matr = np.abs(A)[p[i:],i] / s[p[i:]]
        max_index = np.argmax(matr) + i

        # Updating indicies
        temp = p[i]
        p[i] = p[max_index]
        p[max_index] = temp

    P = np.identity(n)[p]

    # Gauss elimination
    for k in range(0, n-1):
        for i in range(k+1, n):
            m_ik = U[p[i],k] / U[p[k],k]
            L[i,k] += m_ik

            for j in range(k, n):
                U[p[i],j] -= m_ik * U[p[k],j]

    return P, L, U[p]

P, L, U = lu_fac_pivot(A)
print(f'Task 1b\nP = \n{P}\nPA = \n{P@A}\nLU = \n{L@U}')

Task 1b
P = 
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
PA = 
[[ 2.  2.  1.]
 [ 4. -8.  2.]
 [ 5.  0. 10.]]
LU = 
[[ 2.00000000e+00  2.00000000e+00  1.00000000e+00]
 [ 4.00000000e+00 -8.00000000e+00  2.00000000e+00]
 [ 5.00000000e+00 -2.22044605e-16  1.00000000e+01]]


### Oppgave 2

Gitt ligningssystemet 
$$
A \mb{x} = \mb{b}
$$
der $A\in\mathbb{R}^{n\times n}$ og $\mb{b},\mb{x} = \R^{n}$. 

**a)** 
Skriv en funksjon `gauss_naiv` som løser slike ligninger med en naiv Gauss-eliminasjon. 
Skriv ut passende feilmeldinger ved behov. 

*Hint 1:* Bruk `assert` til feilmeldinger. <br>
*Hint 2:* Vær sikker på at $A$ og $\mb{b}$ leses inn som flyttall. <br>
Bruk f.eks. `A = np.array([[....]],dtype=float`). 

**b)**
Skriv en funksjon `gauss_pivot` som bruker løser ligningen med Gauss-eliminasjon med skalert pivotering.

Bruk disse til å løse ligningsystemet $A\mb{x}=\mb{b}$ med

$$
A = \begin{bmatrix} 10 & 0 & 20 & 10 \\
    4 & -9 & 2 & 1 \\8 & 16 & 6 &5 \\ 2 & 3 & 2 & 1
    \end{bmatrix} \qquad 
\mb{b} = \begin{bmatrix} 10 \\ 2 \\ 4 \\ 3 \end{bmatrix}
$$

*Hint 1:* Bruk `assert` til feilmeldinger. <br>
*Hint 2:* Vær sikker på at $A$ og $\mb{b}$ leses inn som flyttall. 
Bruk f.eks. `A = np.array([[....]],dtype=float`). <br>
*Hint 3:* Kontroller gjerne løsningen din ved å bruke `scipy.linalg.solve`. 

**c)** Bruk `scipy.linalg.lu` for å finne $P$, $L$ og $U$ slik at 
$PA=LU$. Får du samme pivot-elementer som du fikk med `gauss_pivot`. 

Del første ligning (første rad i $A$ og $\mb{b}$) med 10. Får du samme pivot-vektor med `gauss_pivot` og  `scipy.linalg.lu` nå?

Forklar det du observerer. 

In [4]:
# Oppgave 2a
def gauss_naive(A_matr, b_vec):
    """
    Solve the equation Ax = b using primitive Gauss Elimination.

    Parameters
    ----------
    A_matr : np.ndarray (N, N)
        Matrix with real elements.
    b_vec : np.ndarray (N,)
        Vector with real elements.

    Returns
    -------
    x : np.ndarray (N,)
        Solution to equation.
    """
    A = np.array(A_matr, copy=True, dtype=float)
    b = np.array(b_vec, copy=True, dtype=float)

    assert A.shape[0] == A.shape[1], 'A is not quadratic'
    assert A.shape[0] == b.shape[0], 'A and b do not have same first dimension'

    x = np.zeros_like(b)
    _, n = A.shape
    L = np.eye(n, n)

    for k in range(0, n-1):
        for i in range(k+1, n):
            m_ik = A[i,k] / A[k,k]
            L[i,k] += m_ik

            for j in range(k, n):
                A[i,j] -= m_ik * A[k,j]

            b[i] -= m_ik * b[k]

    x[-1] = b[-1] / A[-1,-1]
    for i in range(n-2, -1, -1):
        s = 0
        for j in range(i+1, n):
            s += A[i,j] * x[j]
        x[i] = (b[i] - s) / A[i,i]

    return x

# Oppgave 2b
def gauss_pivot(A_matr, b_vec):
    A = np.array(A_matr, copy=True, dtype=float)
    b = np.array(b_vec, copy=True, dtype=float)

    assert A.shape[0] == A.shape[1], 'A is not quadratic'
    assert A.shape[0] == b.shape[0], 'A and b do not have same first dimension'

    x = np.zeros_like(b)
    s = np.max(np.abs(A), axis=1)  # Scaling vector
    n, _ = A.shape
    p = np.arange(n)  # Permutation order vector

    for i in range(n):
        # Finding max element in row
        matr = np.abs(A)[p[i:],i] / s[p[i:]]
        max_index = np.argmax(matr) + i

        # Updating indicies
        temp = p[i]
        p[i] = p[max_index]
        p[max_index] = temp

    for k in range(0, n-1):
        for i in range(k+1, n):
            m_ik = A[p[i],k] / A[p[k],k]

            for j in range(k, n):
                A[p[i],j] -= m_ik * A[p[k],j]

            b[p[i]] -= m_ik * b[p[k]]

    x[-1] = b[p[-1]] / A[p[-1],-1]
    for i in range(n-2, -1, -1):
        s = 0.
        for j in range(i+1, n):
            s += A[p[i],j] * x[j]
        x[i] = (b[p[i]] - s) / A[p[i],i]

    return x

In [5]:
A = np.asarray([
    [10, 0, 20, 10],
    [4, -9, 2, 1],
    [8, 16, 6, 5],
    [2, 3, 2, 1]
], dtype=float)
b = np.asarray([10, 2, 4, 3])

x_anal = solve(A, b)
x_naive = gauss_naive(A, b)
x_pivot = gauss_pivot(A, b)

assert_array_almost_equal(x_anal, x_naive)

print(f'x (scipy.linalg.solve) {x_anal}')
print(f'x (Gauss naive)        {x_naive}')
print(f'x (Gauss pivot)        {x_pivot}')

x (scipy.linalg.solve) [ 1.16666667  0.27777778  2.23611111 -4.63888889]
x (Gauss naive)        [ 1.16666667  0.27777778  2.23611111 -4.63888889]
x (Gauss pivot)        [ 1.16666667  0.27777778  2.23611111 -4.63888889]


### Oppgave 3

Hvilke(n) av de tre matrisene under er strengt diagonal dominant?

$$
A_1 = 
\begin{bmatrix}
4 & 2 & -3 \\ 
2 & 6 & 1 \\
1 & -2 & 6 
\end{bmatrix}, \qquad 
A_2 = 
\begin{bmatrix}
3 & - 1 & 1 \\
2 & -5 & 1 \\
0 & 1 & 2 
\end{bmatrix}, \qquad
A_3 = 
\begin{bmatrix} 
12 & 1 & -11 & 0 \\
3 & 7 & -2 & 1 \\
-2 & 2 & 6 & -1 \\
1 & -1 & 2 & 4
\end{bmatrix}. 
$$

In [6]:
def diagnoal_dominant(A):
    """ Checks if the matrix A is strictly diagnonally dominant. """
    return np.all(2. * np.diag(np.abs(A)) > np.sum(np.abs(A), axis=1))

A1 = np.asarray([
    [4, 2, -3],
    [2, 6, 1],
    [1, -2, 6]
], dtype=float)
print(f'A1 is SDD: {diagnoal_dominant(A1)}')
# Should be False

A2 = np.asarray([
    [3, -1, 1],
    [2, -5, 1],
    [0, 1, 2]
], dtype=float)
print(f'A2 is SDD: {diagnoal_dominant(A2)}')
# Should be True

A3 = np.asarray([
    [12, 1, -11, 0],
    [3, 7, -2, 1],
    [-2, 2, 6, -1],
    [1, -1, 2, 4]
], dtype=float)
print(f'A3 is SDD: {diagnoal_dominant(A3)}')
# Should be False

A1 is SDD: False
A2 is SDD: True
A3 is SDD: False


### Oppgave 4

**(a)**
La $\|\cdot\|_v$ være en vektornorm. Vis at den tilordnede matrisenormen oppfyller
$
\|I\| = 1.
$

**(b)** Frobeniusnormen $$\|A\|_F=\sqrt{\sum_{i=1}^n\sum_{j=1}^n A_{ij}^2}$$ er en populær matrisenorm.
Vis at denne ikke kan være tilordnet noen vektornorm for $n>1$.

**(c)** Kondisjonstallet til en matrise $A$ er definert som $\kappa(A)=\|A\|\,\|A^{-1}\|$.
Hvis vi bruker $\|\cdot\|_p$ som matrisenorm ($p=1,2,\infty$) kan vi tilsvarende definere $\kappa_p(A)$.
Hva er $\kappa_p(D)$ for diagonalmatriser $D$ (sjekk alle nevnte $p$).


**(d)** La $D$ være diagonalmatrisen med diagonalelementer $d_{ii}=\frac{1}{i},\ i=1,\ldots,n$.
Hva er $\kappa_1(D)$ (som funksjon av $n$).


### 4a
Gitt 
$$
||A|| = \text{max}_i
    \left\{
        \frac{||Ax_i||}{||x_i||}
    \right\}
$$
har vi
$$
||I|| = \text{max}_i
    \left\{
        \frac{||Ix_i||}{||x_i||}
    \right\}
    = \text{max}_i
    \left\{
        \frac{||x_i||}{||x_i||}
    \right\}
    = \text{max}_i
    \left\{
        1
    \right\}
    = 1
$$

### 4b
Som vist i (4a) oppfyller en tilordnet matrisenorm $||I|| = 1$. Vi ser at for $n>1$ er
$$
||I||_F = \sqrt{
    \sum_j \sum_i I_{ij}^2
}
= \sqrt{\dim I} > 1
$$

### 4c
Aner ikke

### 4d
$$
\kappa_1(D) = \max_j \sum_i |D_{ij}| \cdot \max_j \sum_i |D^{-1}_{ij}| = 1 \cdot n = n
$$

Sjekker:


In [7]:
D = np.zeros((100, 100), dtype=float)
for i in range(0, 100):
    D[i,i] = 1. / (i+1)
D
print(np.linalg.cond(D))

100.0


**NB!** *De neste oppgavene er frivillige, og skal ikke leveres inn. Men kompetansen som opparbeides ved å gjøre dem inngår i pensum.*

### Oppgave 5

Vi skal studere en interessant familie av matriser som er beryktet for å være dårlig kondisjonerte (stort kondisjonstall). De kalles Hilbertmatriser, er symmetriske, og er definert som

$$
H_{ij} = \frac{1}{i+j-1},\qquad 1\leq i\leq n,\ 1\leq j\leq n.
$$

At de er dårlig kondisjonerte betyr at når vi løser $Hx=b$ så vi en liten endring i $H$ eller $b$ føre til en stor endring i løsningen $x$. Vi gjennomfører et eksperiment for å få litt innsikt i problemet.

Vi kan dra nytte av funksjonen *hilbert* som fins i *scipy.linalg*-biblioteket. Så kan det være nyttig å bruke
*numpy.linalg.cond* som beregner kondisjonstallet til en  matrise.

La oss for ulike verdier av $n$ definere $b\in\mathbb{R}^n$ til å være vectoren med alle elementer lik 1 (lages enkelt med *numpy.ones*. Så lager vi tilfeldige perturbasjoner $b+\delta b$ ved å bruke *numpy.random.rand* (se eksempelkode nedenfor). Vi tar oss friheten å løse ligninger med *numpy* sin innebygde løser *numpy.linalg.solve*.
Bruk alltid $\|\cdot\|_2$ og $\kappa_2(\cdot)$ i denne oppgaven.

**(a)** For hver $n=5, 10, 15, 20$ gjør følgende eksperiment. Løs $Hx=b$ for $x$ med $b$ som beskrevet og lagre $x$.
Lag så en løkke med 1000 eksperimenter der du løser $Hy=b+\delta b$ for $y$ og der du velger $\delta b$ 
med random-generator som beskrevet i eksemplet slik at $\|\delta b\|=0.01\cdot\|b\|$.
Blant alle disse 100 eksperimentene, lagre den maksimale verdien av $\|y-x\|$.
I dette oppsettet kan vi fra timene anslå at

$$
   \frac{\|y-x\|}{\|x\|} \leq \kappa(H)\,\frac{\|\delta b\|}{\|b\|}
$$

Bruk disse lagrede verdiene til estimere $\kappa(H)$, og sammenlign med den virkelige verdien av 
$\kappa(H)$ som du finner fra *numpy.linalg.cond*. 
Skriv ut en tabell der første kolonne er $n$, andre kolonne er estimert kondisjonstall og tredje kolonne er virkelig kondisjonstall.
Sammenlign og kommenter.

**Eksempel:** Med $n=4$, sett $b=[1,1,1,1]^T$ og $\delta b= [0,0.012,-0.016,0]^T$.
Finn for dette spesifikke tilfellet den estimerte $\kappa(H)$.


**(b)** Kanskje du ikke ble imponert over hva kondisjonstallet sa i forrige eksperiment? Det kan i så fall ha å gjøre med at kondisjonstallet gir absolutt verste tilfelle (worst case analysis), mens hvis man har litt "flaks" med valg av $b$ så blir det kanskje ikke så galt. Vi forsøker å fremprovosere en mer kritisk $b$.
Det du skal gjøre er å generere $b$ med en liten algoritme som kan beskrives slik:
Start med $b=[1,\ldots,1]^T$. Lag så en løkke med 20 iterasjoner ('for j in range(20)') der du først
setter $b:=H\cdot b$ og deretter skalerer, $b:=b/\|b\|$.
Gjenta eksperimentet fra **(a)** med dette valget av høyreside $b$ og kommenter resultatet.
Kan du komme opp med en god forklaring på hvorfor $b$ generert på denne måten gir mye større utslag (større estimert kondisjonstall) enn du fikk i **(a)**?

In [8]:
# Litt eksempelkode
import numpy as np
import numpy.linalg as la
from scipy.linalg import hilbert
from scipy.linalg import invhilbert #denne lager eksplisitt den inverse av hilbertmatrisen

# bvec er en liten funksjon som gir oss vector med n 1'ere.
bvec= lambda n: np.ones((n,))
print(bvec(3))
# Kondisjonstallet til 4x4 Hilbertmatrisen
print(la.cond(hilbert(4)))

# Lag en perturbasjon av norm epsilon
def b_perturb(n,epsilon):
    d = 2*np.random.rand(n,1)-1
    d = d*epsilon/la.norm(d,2)
    return d

[1. 1. 1.]
15513.73873892924


In [9]:
# Fyll inn resten koden din her.

### Oppgave 6

La $A,B\in \R^{n \times n}$. Produktet $C=AB$ regnes ut ved følgende algoritme:

$$
\text{for } i=1,\cdots,n \\
\qquad \text{for } j=1,\cdots,n \\
\qquad \qquad c_{ij} = \sum_{k=1}^n a_{ij} b_{kj}. 
$$
 
**a)**

Hvor mange flyttallsoperasjoner krever en slik matrise-matrise multiplikasjon av to $n\times n$-matriser? 

Poenget med resten av denne oppgaven er hovedsaklig å vise hvor effektive Pythons innebygde rutiner for lineær algebra er. 

**b)** Skriv din egen rutine for å regne ut matrise-matrise produktet $C=BA$ med algoritmen over. 

Deretter: Velg $n$. Lag to tilfeldige $n\times n$-matriser $A$ og $B$, Regn ut $C=AB$ med din egen rutine. Ta tiden. Sammenlign med den tiden det tar å gjøre samme operasjon med pythons multiplikasjonsoperator `@` eller `np.dot`. 

Gjør eksperimentet med ulike verdier av $n$, velg $n$ gjerne så stor som du har tålmodighet til. Trekk lærdom av resultatet!

Gjenta gjerne eksperimentet med din egen `gauss_pivot` og `np.linalg.solve`. Velg høyreside selv. 

In [10]:
def mult(A,B):
    ''' 
    Regner ut matrise-matrise produktet C=AB
    '''
    # Fyll inn koden din her

In [11]:
# Lag matrisene
n = 10
A = np.random.randn(n,n)
B = np.random.randn(n,n)

# Test egen kode:
import time
tstart = time.time()
C = mult(A,B)
print(f'Min kode bruker     {time.time()-tstart:.3e} s')

# Test pythons kode
tstart = time.time()
C = A@B
print(f'Pyhtons kode bruker {time.time()-tstart:.3e} s')

Min kode bruker     1.483e-04 s
Pyhtons kode bruker 2.060e-04 s


In [12]:
# Alternativ (og mer nøyaktig metode) for å måle tidsbruk:

%timeit mult(A,B)
%timeit A@B

247 ns ± 48 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
3.38 µs ± 350 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
